In [1]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("D:\car_purchasing.csv")


In [2]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.describe()

In [ ]:
df.drop(
    columns=["customer name", "customer e-mail", "country"],
    inplace=True
)


In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df.drop("car purchase amount",axis=1),df["car purchase amount"],test_size=0.2,random_state=42)

In [ ]:
x_norm_train=sc.fit_transform(x_train)
x_norm_test=sc.transform(x_test)

In [ ]:
lr=LinearRegression()

In [ ]:
x_norm_test

In [ ]:
# 1. Fit the model
lr.fit(x_norm_train, y_train)

# 2. Predict
y_pred = lr.predict(x_norm_test)


In [ ]:
print("R2 :", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


In [ ]:
i = 5   # any index you want

# Predict one sample (must be 2D)
y_pred_one = lr.predict(x_norm_test[i].reshape(1, -1))

# Actual value
y_actual_one = y_test.iloc[i]   # or y_test[i] if numpy array

print("Predicted:", y_pred_one[0])
print("Actual   :", y_actual_one)


In [ ]:
##start with pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, features, label):
        self.features = torch.tensor(
            features, dtype=torch.float32
        )
        self.label = torch.tensor(
            label.values, dtype=torch.long
        )

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        return self.features[idx], self.label[idx]


In [ ]:
train_dataset=CustomDataset(x_norm_train,y_train)
test_dataset=CustomDataset(x_norm_test,y_test)

In [ ]:
train_loader=DataLoader(train_dataset,batch_size=5,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=5,shuffle=False)

In [ ]:
class mynn(nn.module):
  def__init__(self,num_features): 
  super().init__()
  self.model=nn.Sequential(
    nn.Linear(num_features,128),
    nn.Relu(),
    nn.Linear(128,64),
    nn.Relu(),
    nn.Linear(64,1)
    )
  def foward(self,x):
    return self.model(x)


In [3]:
##set learning rate
epochs=100
learning_rate=0.001

In [4]:
##intiate the model
model=mynn(x_norm_train.shape[1])
##loss function
criterion = nn.MSELoss()
##optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)


NameError: name 'mynn' is not defined

In [ ]:
##training loop

for epoch in range(epochs):

        total_epoch_loss=0

    for batch_features,batch_label in train_loader:

        #forward pass
        outputs=model(batch_features)
        #calculate loss
        loss=criterion(outputs,batch_label)

        #back pass
        optimizer.zero_grad()
        loss.backward()
        ##update grad
        optimizer.step()

        total_epoch_loss=total_epoch_loss+loss.item()
        avg_loss=total_epoch_loss/len(train_loader)

        print(avg_loss)


In [ ]:
##set model to eval mode
model.eval()

In [ ]:
total_loss = 0.0
total_samples = 0

with torch.no_grad():      # 2️⃣ disable gradients
    for batch_features, batch_label in test_loader:

        # 3️⃣ forward pass
        outputs = model(batch_features)

        # 4️⃣ compute batch loss
        loss = criterion(outputs, batch_label)

        # 5️⃣ accumulate weighted loss
        batch_size = batch_features.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

# 6️⃣ dataset-level mean loss
avg_loss = total_loss / total_samples

print("Test MSE:", avg_loss)